# Trabajo Practico 2 - Machine Learning

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model

from sklearn.metrics import mean_absolute_error

# Importamos los datos de entrenamiento

In [2]:
train = pd.read_csv('./train.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":'category'
            ,"fecha":str 
            ,"ciudad": 'category'
            ,"provincia": 'category'
            ,"tipodepropiedad": 'category' })

# Importamos los datos de Test

Sobre este set de datos no tengo la columna 'precio' que es justamento lo que quiero predecir

In [3]:
test = pd.read_csv('./test.csv'
            ,dtype={
             "id":np.int32
            ,"titulo":'category'
            ,"fecha":str 
            ,"ciudad": 'category'
            ,"provincia": 'category'
            ,"tipodepropiedad": 'category' })

El formato del Submit es ( Id , Precio ) los ids son los que estan en el set de Test y el Precio el que predice mi modelo

 # Métrica: Root Mean Squared Logarithmic Error

In [4]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [5]:
#Utilizaremos MAE como métrica para esta competencia.
dummy_rmsle = mean_absolute_error([5], [0])
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

MAE Error (Test): 5.00000


# 1º - Modelo: Constante

In [6]:
# Tiramos todas las columnas no numéricas para que sklearn nos acepte el dataframe. 
# Igualmente, el DummyRegressor no va a utilizar ninguna más que el precio 
drop_cols = ['fecha', 'ciudad', 'idzona', 'tipodepropiedad', 'provincia', 'titulo', 'descripcion', 'direccion']
X = train.drop(['precio'] + drop_cols, axis=1)
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 14) y=(180000,)
Test  shapes: X=(60000, 14)  y=(60000,)


In [7]:
# Create a dummy regressor
dummy_constant  = DummyRegressor(strategy='constant', constant=500)

# "Train" dummy regressor ( Aca ya estoy listo para predecir)
dummy_constant.fit(X_train[['id']], y_train)

DummyRegressor(constant=array(500), quantile=None, strategy='constant')

In [8]:
#Predigo valores Puntales modo de ejemplo
dummy_constant.predict(np.array([[100]]))

array([500])

In [9]:
#Calculo de Errores contra mi set de test
pred = dummy_constant.predict(    test[['id']]     )

#dummy_rmsle = mean_absolute_error(y_test, pred)
#dummy_rmsle_train = mean_absolute_error(y_train, dummy_constant.predict(X_train) )

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, dummy_constant.predict(X_train) )
dummy_rmsle = RMSLE(y_test, pred)

print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, dummy_constant.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 8.23876
RMSLE Error (Test): 8.23005
MAE Error (train): 2536413.14206
MAE Error (Test): 2512113.95740


# 2º Modelo Promedio

In [10]:
# Create a dummy regressor
dummy_mean = DummyRegressor(strategy='mean')

# "Train" dummy regressor
dummy_mean.fit(X_train, y_train)

DummyRegressor(constant=None, quantile=None, strategy='mean')

In [11]:
#Calculo de Errores contra mi set de test
pred = dummy_mean.predict(    test[['id']]     )

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle = RMSLE(y_test, pred)
dummy_rmsle_train = RMSLE(y_train, dummy_mean.predict(X_train) )

print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, dummy_mean.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90228
RMSLE Error (Test): 0.90318
MAE Error (train): 1612604.48000
MAE Error (Test): 1602549.96274


In [12]:
#Exporto csv con la prediccion
pred = dummy_mean.predict(test) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

# 3º Modelo Regresion Lineal 

### metroscubiertos

In [13]:
X = train[['metroscubiertos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
#print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
#print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle = RMSLE(y_test, pred)
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.65673
RMSLE Error (Test): 0.65648
MAE Error (train): 1202603.98514
MAE Error (Test): 1191062.51364


In [ ]:
#Exporto csv con la prediccion
pred = reg.predict(test['metroscubiertos'].to_frame().fillna(X_test.mean())   ) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

### metrostotales

In [16]:
X = train[['metrostotales']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 1) y=(180000,)
Test  shapes: X=(60000, 1)  y=(60000,)


In [17]:
#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [18]:
#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.77069
RMSLE Error (Test): 0.76986
MAE Error (train): 1384485.13818
MAE Error (Test): 1375006.68201


### baños 

In [20]:
X = train[['banos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
print(f"Train shapes: X={X_train.shape} y={y_train.shape}")
print(f"Test  shapes: X={X_test.shape}  y={y_test.shape}")

Train shapes: X=(180000, 1) y=(180000,)
Test  shapes: X=(60000, 1)  y=(60000,)


In [21]:
#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [22]:
#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)
#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.71305
RMSLE Error (Test): 0.71311
MAE Error (train): 1326576.26867
MAE Error (Test): 1316655.85210


### garages

In [24]:
X = train[['garages']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.84448
RMSLE Error (Test): 0.84314
MAE Error (train): 1505488.62183
MAE Error (Test): 1492386.44838


### habitaciones

In [28]:
X = train[['habitaciones']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.85682
RMSLE Error (Test): 0.85757
MAE Error (train): 1549433.63003
MAE Error (Test): 1541163.97837


### gimnasio

In [29]:
X = train[['gimnasio']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89044
RMSLE Error (Test): 0.89150
MAE Error (train): 1587871.29864
MAE Error (Test): 1579081.68876


### usosmultiples

In [30]:
X = train[['usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89135
RMSLE Error (Test): 0.89216
MAE Error (train): 1590987.90271
MAE Error (Test): 1580698.12959


### piscina

In [31]:
X = train[['piscina']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89413
RMSLE Error (Test): 0.89515
MAE Error (train): 1598374.73603
MAE Error (Test): 1588343.97441


### longitud del titulo

In [32]:
train['tituloLen'] = train['titulo'].apply(len)
X = train[['tituloLen']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90225
RMSLE Error (Test): 0.90318
MAE Error (train): 1612614.37721
MAE Error (Test): 1602586.86903


### tipo de propiedad

In [33]:

a = train

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a.replace(cleanup_nums, inplace=True)


X = a[['tipodepropiedad']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.90122
RMSLE Error (Test): 0.90258
MAE Error (train): 1611042.38531
MAE Error (Test): 1602328.47233


### fecha : año

In [34]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year

X = a[['anio']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.89211
RMSLE Error (Test): 0.89394
MAE Error (train): 1597245.39542
MAE Error (Test): 1588343.37661


# 4º Modelo : Regresion Lineal Multiple

### 1

In [36]:
X = train[['metrostotales','metroscubiertos','banos']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.62025
RMSLE Error (Test): 0.61993
MAE Error (train): 1161884.23342
MAE Error (Test): 1152018.48875


### 2   

In [37]:
X = train[['metrostotales','metroscubiertos','banos','habitaciones','gimnasio','usosmultiples','piscina']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.60806
RMSLE Error (Test): 0.60794
MAE Error (train): 1129269.61449
MAE Error (Test): 1119327.35026


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
b = test[['metrostotales','metroscubiertos','banos','habitaciones','gimnasio','usosmultiples','piscina']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = reg.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 3

In [38]:
X = train[['metrostotales','metroscubiertos','banos','gimnasio','habitaciones','antiguedad']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.60840
RMSLE Error (Test): 0.60735
MAE Error (train): 1124385.30383
MAE Error (Test): 1114935.52215


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


## 4

In [39]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month

X = a[['anio','mes','metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

#Creo y Entreno el modelo de regresion Lineal
reg = linear_model.LinearRegression()
reg.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = reg.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, reg.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, reg.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.61060
RMSLE Error (Test): 0.60723
MAE Error (train): 1105706.21188
MAE Error (Test): 1097307.19302


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


# 5º Modelo : Random Forest Regresion

In [57]:
# Load libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets

## 1 (Score 716812.39431)

In [47]:
# Load data with only two features

X = train[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0, n_jobs=-1,max_depth = 20,min_samples_split =20 ,n_estimators =50)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)


#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle = RMSLE(y_test, pred)
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )

print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")



RMSLE Error (train): 0.33201
RMSLE Error (Test): 0.41936
MAE Error (train): 559334.94425
MAE Error (Test): 718305.31512


In [ ]:
b = test[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 2   ( score 702024.74347 ) 

In [48]:
# Load data with only two features

X = train[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0, n_jobs=-1,max_depth = 100,max_features =6,min_samples_split = 4 ,n_estimators =200)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.22532
RMSLE Error (Test): 0.40942
MAE Error (train): 337653.34109
MAE Error (Test): 704304.79830


In [ ]:
b = test[['metrostotales','centroscomercialescercanos','metroscubiertos','banos','habitaciones','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 3 (Score 669492.31492) 

In [58]:
a = train
a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek

# Load data with only two features

X = a[['anio','mes','metrostotales','centroscomercialescercanos','piscina','escuelascercanas','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,max_depth = 100,max_features = 8 ,min_samples_split = 2 ,n_estimators =99)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.12171
RMSLE Error (Test): 0.39234
MAE Error (train): 150703.83197
MAE Error (Test): 659485.92929


In [ ]:
c = test
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month
c['dia'] = c['fecha'].dt.dayofweek

b = c[['anio','mes','metrostotales','centroscomercialescercanos','piscina','escuelascercanas','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

## 4  score 583658.64804

In [59]:

a = train

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = a.replace(cleanup_nums)
#a['tituloLen'] = a['titulo'].apply(len)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month
a['dia'] = a['fecha'].dt.dayofweek

a = a.fillna(X_train.mean())

# Load data with only two features

X = a[['tipodepropiedad','anio','metrostotales','antiguedad','centroscomercialescercanos','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,
                             max_depth = 100,max_features = 5 ,min_samples_split = 2 ,
                             n_estimators = 400)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")


dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")


RMSLE Error (train): 0.13892
RMSLE Error (Test): 0.35932
MAE Error (train): 183518.00384
MAE Error (Test): 599651.31538


In [60]:
X_train.mean() # Valores en los Nulls

tipodepropiedad                    1.87
anio                            2014.67
metrostotales                    176.92
antiguedad                         8.09
centroscomercialescercanos         0.40
garages                            1.55
piscina                            0.09
metroscubiertos                  174.16
banos                              2.13
habitaciones                       2.90
lat                               20.70
lng                              -99.53
idzona                       2438952.04
gimnasio                           0.06
usosmultiples                      0.06
dtype: float64

In [63]:
c = test
cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

c = c.replace(cleanup_nums)
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year

c = c.fillna(X_train.mean())

b = c[['tipodepropiedad','anio','metrostotales','antiguedad','centroscomercialescercanos','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
b = b.fillna(b.mean())

#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)


MAE Error (train): 183518.00384
MAE Error (Test): 2068041.14192


## 5

In [64]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=3)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create decision tree classifer object
regr = RandomForestRegressor(random_state=0,bootstrap =False,min_samples_leaf=2 ,n_jobs=-1,max_depth = 80,max_features = 10 ,min_samples_split = 2 ,n_estimators = 500)

# Train model
model = regr.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.09347
RMSLE Error (Test): 0.35775
MAE Error (train): 112026.05990
MAE Error (Test): 591196.32856


In [ ]:

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

c = test
c = c.replace(cleanup_nums)
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month

drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
c = c.drop(drop_cols, axis=1)

#Relleno los nulls
c[['tipodepropiedad']] = c[['tipodepropiedad']].fillna(value=3)
c[['metroscubiertos']] = c[['metroscubiertos']].fillna(value= (c['metroscubiertos'].mean())  )
c[['antiguedad']] = c[['antiguedad']].fillna(value= (c['antiguedad'].mean()) )
c[['habitaciones']] = c[['habitaciones']].fillna(value= (c['habitaciones'].mean()) )
c[['banos']] = c[['banos']].fillna(value= (c['habitaciones'].mean()))
c[['idzona']] = c[['idzona']].fillna(value=(c['idzona'].mean()))
c[['garages']] = c[['garages']].fillna(value=(c['garages'].mean()))
c[['metrostotales']] = c[['metrostotales']].fillna(value=(c['metrostotales'].mean()))
c[['lng']] = c[['lng']].fillna(value=(c['lng'].mean()))
c[['lat']] = c[['lat']].fillna(value=(c['lat'].mean()))


b = c[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]


#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

# 6º -Modelo : KNN

In [66]:
from sklearn.neighbors import KNeighborsRegressor

## 1  Score:  745536.27245

In [67]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=1)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Create KNN Regressor
regressor = KNeighborsRegressor(20)
#cross_val_score(regressor, boston.data, boston.target, cv=10)

# Train model
model = regressor.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.42322
RMSLE Error (Test): 0.44397
MAE Error (train): 729996.35736
MAE Error (Test): 762946.37406


In [ ]:
cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

c = test
c = c.replace(cleanup_nums)
c['fecha'] = pd.to_datetime(c['fecha'])
c['anio'] = c['fecha'].dt.year
c['mes'] = c['fecha'].dt.month

drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
c = c.drop(drop_cols, axis=1)

#Relleno los nulls
c[['tipodepropiedad']] = c[['tipodepropiedad']].fillna(value=3)
c[['metroscubiertos']] = c[['metroscubiertos']].fillna(value= (c['metroscubiertos'].mean())  )
c[['antiguedad']] = c[['antiguedad']].fillna(value= (c['antiguedad'].mean()) )
c[['habitaciones']] = c[['habitaciones']].fillna(value= (c['habitaciones'].mean()) )
c[['banos']] = c[['banos']].fillna(value= (c['habitaciones'].mean()))
c[['idzona']] = c[['idzona']].fillna(value=(c['idzona'].mean()))
c[['garages']] = c[['garages']].fillna(value=(c['garages'].mean()))
c[['metrostotales']] = c[['metrostotales']].fillna(value=(c['metrostotales'].mean()))
c[['lng']] = c[['lng']].fillna(value=(c['lng'].mean()))
c[['lat']] = c[['lat']].fillna(value=(c['lat'].mean()))


b = c[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]


#Exporto csv con la prediccion
pred = model.predict(b) 
#Creo un dataframe con el formato ( id , precio (El precio predecido ))
res = test['id'].to_frame()
res.insert(1,'target', pred , True) 
#Exportamos la prediccion en formato csv sin labels
res.to_csv(r'C:\Users\Giova\JupiterLabFolder\Archivos CSV\res.csv',index=False , header=True)

# 7º Modelo : Ridge Regression

In [68]:
from sklearn.linear_model import Ridge

In [70]:



#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=1)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Creo Rigge
clf = Ridge(alpha=1.0 , fit_intercept = False , max_iter =1000)

# Train model
model = clf.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

RMSLE Error (train): 0.61568
RMSLE Error (Test): 0.61833
MAE Error (train): 1084562.35267
MAE Error (Test): 1075626.77831


C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Giova\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


# 8º Modelo: Support Vector Machines

In [72]:
from sklearn import svm

In [ ]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=1)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['tipodepropiedad','anio','mes','metrostotales','antiguedad','garages','piscina','metroscubiertos','banos','habitaciones','lat','lng','idzona','gimnasio','usosmultiples']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Creo 
clf = svm.SVR()

# Train model
model = clf.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")

C:\Users\Giova\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# 9º Modelo : PLSRegression ( Memory Error)

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=1)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['metroscubiertos']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Creo 
pls2 = PLSRegression(n_components=1)

# Train model
model = pls2.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

# 10 º Modelo : NaiveBayes

In [7]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
#Preparo los datos

cleanup_nums = {"tipodepropiedad":     {
    "Casa": 1,
    "Apartamento": 2, 
    "Casa en condominio": 3,
    "Terreno": 4,
    "Local Comercial": 5,
    "Oficina comercial": 6,
    "Bodega comercial": 7,
    "Edificio": 8,
    "Terreno comercial": 9,
    "Casa uso de suelo": 10,
    "Quinta Vacacional": 11,
    "Duplex": 12,
    "Villa": 13,
    "Inmuebles productivos urbanos": 14,
    "Rancho": 15,
    "Local en centro comercial": 16,
    "Departamento Compartido": 17,
    "Otros": 18,
    "Nave industrial": 19,
    "Terreno industrial": 20,
    "Huerta": 21,
    "Lote": 22,
    "Hospedaje": 23,
    "Garage": 24
                                   
}  }

a = train
a = a.replace(cleanup_nums)

a['fecha'] = pd.to_datetime(a['fecha'])
a['anio'] = a['fecha'].dt.year
a['mes'] = a['fecha'].dt.month


drop_cols = [ 'ciudad', 'provincia', 'titulo', 'descripcion', 'direccion']
a = a.drop(drop_cols, axis=1)

#Relleno los nulls
a[['tipodepropiedad']] = a[['tipodepropiedad']].fillna(value=1)
a[['metroscubiertos']] = a[['metroscubiertos']].fillna(value= (a['metroscubiertos'].mean())  )
a[['antiguedad']] = a[['antiguedad']].fillna(value= (a['antiguedad'].mean()) )
a[['habitaciones']] = a[['habitaciones']].fillna(value= (a['habitaciones'].mean()) )
a[['banos']] = a[['banos']].fillna(value= (a['habitaciones'].mean()))
a[['idzona']] = a[['idzona']].fillna(value=(a['idzona'].mean()))
a[['garages']] = a[['garages']].fillna(value=(a['garages'].mean()))
a[['metrostotales']] = a[['metrostotales']].fillna(value=(a['metrostotales'].mean()))
a[['lng']] = a[['lng']].fillna(value=(a['lng'].mean()))
a[['lat']] = a[['lat']].fillna(value=(a['lat'].mean()))


# Load data with only two features

X = a[['metroscubiertos']]
y = a['precio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

#Elimino los Nulls en los x de Entrenaniento y cuando voy a testear
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Creo 
clf = GaussianNB()

# Train model
model = clf.fit(X_train, y_train)

#Calculo de Errores contra mi set de test
pred = model.predict(X_test)
dummy_rmsle = RMSLE(y_test, pred)

#Calculo de Errores contra mi set de entrenamiento
dummy_rmsle_train = RMSLE(y_train, model.predict(X_train) )
print(f"RMSLE Error (train): {dummy_rmsle_train:.5f}")
print(f"RMSLE Error (Test): {dummy_rmsle:.5f}")

dummy_rmsle = mean_absolute_error(y_test, pred)
dummy_rmsle_train = mean_absolute_error(y_train, model.predict(X_train) )

print(f"MAE Error (train): {dummy_rmsle_train:.5f}")
print(f"MAE Error (Test): {dummy_rmsle:.5f}")